In [14]:
from pygalfitm import PyGalfitm
from pygalfitm.VOs import splus
from pygalfitm.auxiliars import string_times_x, get_dims, get_exptime, unpack_file
from pygalfitm.psf import make_psf

import pandas as pd
import splusdata

import os

In [2]:
import pygalfitm

pygalfitm.__path__

['/Users/gustavo/miniconda3/lib/python3.9/site-packages/pygalfitm']

In [3]:
name = "ari_test"

ra = 52.430082897775755	
dec = -33.557141932864596
cut_size = 200
bands = ["I", "R", "G"]

DATA_FOLDER = "data/"
OUTPUT_FOLDER = "outputs/"

In [4]:
conn = splusdata.connect("gustavo", "asdflkjh")

You have access to internal data


In [5]:
pyg = PyGalfitm() 

In [8]:
import requests

def check_vo_file(file, download_link):
    if not os.path.exists(os.path.join(pygalfitm.__path__[0], f'{file}')):
        print("Downloading " + file)
        r = requests.get(download_link)
        print("Writing " + os.path.join(pygalfitm.__path__[0], file))
        open(os.path.join(pygalfitm.__path__[0], file), "wb").write(r.content)
        print("Done!")

In [13]:
r.content



175683

In [71]:
import pandas as pd
from astropy.io.fits import getheader

from pygalfitm import PyGalfitm
from pygalfitm.VOs import splus
from pygalfitm.auxiliars import string_times_x, get_dims, get_exptime, unpack_file
from pygalfitm.psf import make_psf

def get_splus(ra, dec, cut_size, data_folder, output_folder, bands = ["I", "R", "G"], zpfile = None,
    SPLUS_WAVELENGHTS = {
            "i": 7670.59, 
            "r": 6251.83, 
            "g": 4758.49, 
            "z": 8936.64,
            "u": 3533.29, 
            "J0378": 3773.13, 
            "J0395": 3940.70, 
            "J0410": 4095.27, 
            "J0430": 4292.39,
            "J0515": 5133.15,
            "J0660": 6613.88,
            "J0861": 8607.59
    }):


    ##Get wavelenghts, input_images, psf_images, filters, zps
    wavelenghts = ""
    input_images = ""
    psf_images = ""
    filters = ""
    zps = ""

    for band in bands:
        band = band.lower()
        try:
            conn.get_cut(ra, dec, 200, band.upper(), filepath=os.path.join(data_folder, f'{name}_{band.lower()}.fits'))
        except Exception as e:
            print(e)
        
        try:
            unpack_file(os.path.join(data_folder, f'{name}_{band.lower()}.fits.fz'))
        except Exception as e:
            print(e)
            print("Make sure you have fpack (cfitsio) in your system alias.")
        
        make_psf(os.path.join(data_folder, f'{name}_{band.lower()}.fits'), outfile=os.path.join(data_folder, f'psf_{name}_{band.lower()}.fits'))

        input_images += "," + os.path.join(data_folder, f'{name}_{band.lower()}.fits')
        psf_images += "," + os.path.join(data_folder, f'psf_{name}_{band.lower()}.fits')
        filters += "," + str(band).lower()
        wavelenghts += "," + str(SPLUS_WAVELENGHTS[band.lower().replace("f", "J0")])

    input_images = input_images[1:]
    psf_images = psf_images[1:]
    filters = filters[1:]    
    wavelenghts = wavelenghts[1:]


    ## Get ZPs
    check_vo_file("VOs/splusZPs.csv", "https://splus.cloud/files/documentation/iDR4/tabelas/iDR4_zero-points.csv") ## Check if zps file exists
    df = pd.read_csv(os.path.join(pygalfitm.__path__[0], "VOs/splusZPs.csv"))
    header = getheader(os.path.join(data_folder, name + f"_{bands[0].lower()}.fits"))
    field = header['OBJECT']

    for band in bands:
        zps += "," + str(df[df['Field'] == field.replace("_", "-")][f'ZP_{band.lower().replace("f", "J0")}'].values[0])
    zps = zps[1:]

    ## Get axis_ratios, effective_rs, position_angles, mags
    axis_ratios : str = ""
    effective_rs : str = ""
    position_angles : str = ""
    mags : str = ""

    for band in bands:
        band = band.lower()

        table = conn.query(f"""
            select * from "idr4_single"."idr4_single_{band}" as x
            where 
            1 = CONTAINS( POINT('ICRS', x.ra_{band}, x.dec_{band}), 
            CIRCLE('ICRS', {ra}, {dec}, 0.0015))
        """)

        print(f"""
            select * from "idr4_single"."idr4_single_{band}" as x
            where 
            1 = CONTAINS( POINT('ICRS', x.ra_{band}, x.dec_{band}), 
            CIRCLE('ICRS', {ra}, {dec}, 0.0015))
        """, table)
        
        axis_ratios += "," + str( table[0][f"B_{band}"]/table[0][f"A_{band}"] )
        effective_rs += "," + str( table[0][f"FLUX_RADIUS_50_{band}"] )
        position_angles += "," + str( table[0][f"THETA_{band}"] )
        mags += "," + str( table[0][f"{band}_auto"]  )

    axis_ratios = axis_ratios[1:]
    effective_rs = effective_rs[1:]
    position_angles = position_angles[1:]
    mags = mags[1:]


    ## Setup pygalfitm class
    pyg.activate_components()
    pyg.activate_components(["sersic"])

    pyg.set_base({
        "A": input_images,
        "A1": filters,
        "B": os.path.join(output_folder, name + "ss.fits"),
        "C": "none",
        "D": psf_images,
        "A2": wavelenghts,
        "H": f"1   {cut_size}  1   {cut_size}",
        "I": f"{cut_size} {cut_size}",
        "J": zps,
        "K": "0.55 0.55"
    })

    pyg.set_component("sersic", {
        "1": ( string_times_x(cut_size / 2, 3), 1, "band" ),
        "2": ( string_times_x(cut_size / 2, 3), 1, "band" ), 
        "3": ( mags, 3, "band" ), 
        "4": ( effective_rs, 2, "band" ),
        "5": ( string_times_x("4", 3), 2, "band" ),
        "9": ( axis_ratios, 1, "band" ),
        "10": ( position_angles, 1, "band" )
    })

    pyg.write_feedme()
    _ = pyg.run()
    print(_)

In [73]:
get_splus(ra, dec, 200, 'data/', 'outputs/')

File data/ari_test_i.fits.fz already exists. If you mean to replace it then use the argument "overwrite=True".
File data/ari_test_r.fits.fz already exists. If you mean to replace it then use the argument "overwrite=True".
File data/ari_test_g.fits.fz already exists. If you mean to replace it then use the argument "overwrite=True".
finished

            select * from "idr4_single"."idr4_single_i" as x
            where 
            1 = CONTAINS( POINT('ICRS', x.ra_i, x.dec_i), 
            CIRCLE('ICRS', 52.430082897775755, -33.557141932864596, 0.0015))
             A_i      BACKGROUND_i     B_i      ... THRESHOLD_i    X_i       Y_i   
------------ ------------ ------------ ... ----------- --------- ---------
 0.003166629  0.029616702  0.002037362 ...  0.21515957 10101.477   5437.62
0.0030444143  0.023557786 0.0021432189 ...  0.19112483 1295.3931 5440.7197
finished

            select * from "idr4_single"."idr4_single_r" as x
            where 
            1 = CONTAINS( POINT('ICRS', x.

CalledProcessError: Command './new_tests/galfitm-1.4.4-osx galfit.feedme' returned non-zero exit status 127.

In [19]:
conn.get_cut(ra, dec, 200, 'R')

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x16b6f2370>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x16b6f2c40>]

In [53]:
df[df["Field"] == "STRIPE82-0002"]["ZP_i"]

1460    23.412
Name: ZP_i, dtype: float64

In [51]:
df1 = pd.read_csv(os.path.join(pygalfitm.__path__[0], "VOs/splusZPs.csv"))

In [52]:
df1

,Field,RA,DEC,ZP_u,ZP_J0378,ZP_J0395,ZP_J0410,ZP_J0430,ZP_g,ZP_J0515,ZP_r,ZP_J0660,ZP_i,ZP_J0861,ZP_z
0,HYDRA-0011,150.77,-23.91,19.978,19.217,19.283,20.346,20.511,22.781,20.719,22.710,20.207,22.488,20.757,21.904
1,HYDRA-0012,152.31,-23.91,20.824,20.049,20.073,21.092,21.220,23.569,21.506,23.573,21.096,23.361,21.624,22.781
2,HYDRA-0013,153.85,-23.91,20.850,20.058,20.069,21.108,21.225,23.569,21.502,23.558,21.084,23.344,21.606,22.771
3,HYDRA-0014,155.38,-23.91,20.775,19.981,19.994,20.990,21.087,23.436,21.376,23.478,21.006,23.288,21.579,22.685
4,HYDRA-0015,156.93,-23.91,20.903,20.074,20.070,21.112,21.256,23.589,21.505,23.525,20.982,23.303,21.389,22.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,STRIPE82-0166,355.75,0.70,20.908,19.969,19.977,21.106,21.199,23.553,21.126,23.527,21.082,23.342,21.619,22.732
1625,STRIPE82-0167,357.17,-0.70,20.096,19.312,19.408,20.494,20.669,22.914,20.792,22.779,20.267,22.547,20.807,21.974
1626,STRIPE82-0168,357.17,0.70,20.903,20.110,20.076,21.147,21.275,23.616,21.554,23.598,21.119,23.401,21.655,22.843
1627,STRIPE82-0169,358.58,-0.70,20.139,19.411,19.531,20.551,20.690,22.927,20.791,22.797,20.249,22.569,20.757,22.034
